In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [2]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'

In [3]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 64
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10


In [4]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    classes=['normal','pneumonia'],
    class_mode='binary',
    subset='training'
)

Found 9534 images belonging to 2 classes.


In [5]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal','pneumonia'],
    subset='training'
)

Found 1490 images belonging to 2 classes.


In [6]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['normal','pneumonia'],
        #subset='validation'
)

Found 1489 images belonging to 2 classes.


In [24]:
import tensorflow as tf
from keras.utils import np_utils
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import kerastuner as kt
from keras.layers import Convolution2D,Flatten,Dense,Activation


In [31]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

# build a sequential model
model = Sequential()
model.add(InputLayer(input_shape=(224, 224, 3)))

# 1st conv block
model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=1, activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()
# fit on data for 30 epochs
#model.fit_generator(train, epochs=30, validation_data=val)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 224, 224, 25)      1900      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 25)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 50)        31300     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 28, 28, 50)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 50)        200       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 70)        31570     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 70)         

In [32]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
148/148 [==============================] - 587s 4s/step - loss: 0.4902 - accuracy: 0.7688 - val_loss: 1.0751 - val_accuracy: 0.4531
Epoch 2/5
148/148 [==============================] - 508s 3s/step - loss: 0.3226 - accuracy: 0.8694 - val_loss: 0.6206 - val_accuracy: 0.6929
Epoch 3/5
148/148 [==============================] - 530s 4s/step - loss: 0.2867 - accuracy: 0.8845 - val_loss: 0.7123 - val_accuracy: 0.6719
Epoch 4/5
148/148 [==============================] - 509s 3s/step - loss: 0.2910 - accuracy: 0.8869 - val_loss: 2.4463 - val_accuracy: 0.4266
Epoch 5/5
148/148 [==============================] - 515s 3s/step - loss: 0.2673 - accuracy: 0.8977 - val_loss: 0.4315 - val_accuracy: 0.8397


In [33]:
model.save('F:/Xray_Gaus/CNN_binary_wt')

history.history

INFO:tensorflow:Assets written to: F:/Xray_Gaus/CNN_binary_wt\assets


{'loss': [0.4168108403682709,
  0.322676420211792,
  0.2978545129299164,
  0.2832720875740051,
  0.27223294973373413],
 'accuracy': [0.8159450888633728,
  0.8710665106773376,
  0.8816261887550354,
  0.8895459175109863,
  0.8942977786064148],
 'val_loss': [1.075124740600586,
  0.6205593943595886,
  0.7122825384140015,
  2.4463088512420654,
  0.43149393796920776],
 'val_accuracy': [0.453125,
  0.6929348111152649,
  0.671875,
  0.426630437374115,
  0.8396739363670349]}

In [34]:
results = model.evaluate(test_gen, batch_size=128)

47/47 [==============================] - 39s 844ms/step - loss: 0.3263 - accuracy: 0.8899


In [38]:
y_pred = model.predict(test_gen, verbose=1)
#y_pred = np.argmax(y_pred, axis = 1)

#y_pred=model.predict(X_test)

for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

47/47 [==============================] - 18s 376ms/step


In [39]:
y_true=test_gen.labels

In [40]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.61      0.67      0.64       885
           1       0.44      0.38      0.40       604

    accuracy                           0.55      1489
   macro avg       0.52      0.52      0.52      1489
weighted avg       0.54      0.55      0.54      1489



In [22]:
'''
def create_model(hyperparam):
    model = keras.models.Sequential()

    model.add(Convolution2D(
    filters=hyperparam.Int('convolution_1',min_value=32, max_value=128, step=16), kernel_size=hyperparam.Choice('convolution_1', values     = [3,6]),
    activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)
                            )),

    model.add(Convolution2D(
    filters=hyperparam.Int('convolution_2', min_value=64, max_value=128, step=16),kernel_size=hyperparam.Choice('convolution_2', values     = [3,6]),
    activation='relu'
                            )),

    model.add(Convolution2D(
    filters=hyperparam.Int('convolution_3', min_value=64, max_value=128, step=16),kernel_size=hyperparam.Choice('convolution_3', values = [3,6]),
    activation='relu'
                            )),

    model.add(Flatten()),
    model.add(Dense(1)),
    model.add(Activation("softmax")),
    model.compile(optimizer=keras.optimizers.Adam(hyperparam.Choice('learning_rate', values=[1e-2, 1e-3])),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
    return model '''

In [25]:
'''tuner_search=RandomSearch(create_model,
                       objective='val_accuracy',
                       max_trials=5,directory='output',project_name="Med")
'''

In [26]:
tuner_search.search(train_gen,epochs=5,validation_data=val_gen)


es\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\mprad\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py", line 223, in add
    output_tensor = layer(self.outputs[0])
  File "C:\Users\mprad\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py", line 951, in __call__
    return self._functional_construction_call(inputs, args, kwargs,
  File "C:\Users\mprad\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py", line 1090, in _functional_construction_call
    outputs = self._keras_tensor_symbolic_call(
  File "C:\Users\mprad\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py", line 822, in _keras_tensor_symbolic_call
    return self._infer_output_signature(inputs, args, kwargs, input_masks)
  File "C:\Users\mprad\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py", line 863, in _infer_output_sign

RuntimeError: Too many failed attempts to build model.